# Extraemos los datos de google maps

In [ ]:
import requests
import json
import time
from google.colab import drive

# =========================================
# MONTAR GOOGLE DRIVE
# =========================================

# Carpeta donde quieres guardar el JSON
RUTA_DRIVE = "/content/drive/MyDrive/LibreriasEcuador/"
!mkdir -p "{RUTA_DRIVE}"  # Crear carpeta si no existe

# =========================================
# CONFIGURACIÓN PRINCIPAL
# =========================================
API_KEY = "TU_API_KEY_AQUI"  # Reemplaza con tu API Key de Google Places
RADIUS_M = 50000  # 50 km
PAUSA_REQUEST = 0.2

# Palabras clave más relevantes
KEYWORDS = [
    "librería",
    "venta de libros",
    "libros",
    "bookstore",
    "book shop",
    "papelería",
    "material educativo",
    "textos escolares",
    "libros escolares",
    "editorial"
]

# Provincias seleccionadas con coordenadas centrales
PROVINCES = {
    "Pichincha": (-0.2100, -78.5000),
    "Pastaza": (-1.4667, -78.0000)
}

# Grilla de 3 puntos por provincia (aproximación: centro + norte + sur)
GRID_OFFSETS = [
    (0.0, 0.0),   # centro
    (0.3, 0.3),   # noreste
    (-0.3, -0.3)  # suroeste
]

# =========================================
# FUNCIÓN: Nearby Search
# =========================================
def buscar_lugares(lat, lng, keyword):
    url = (
        f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
        f"location={lat},{lng}&radius={RADIUS_M}&keyword={keyword}&key={API_KEY}"
    )
    resultados = []

    while True:
        response = requests.get(url)
        data = response.json()

        if "results" in data:
            resultados.extend(data["results"])

        if "next_page_token" in data:
            time.sleep(2)  # recomendado por Google
            token = data["next_page_token"]
            url = (
                f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
                f"pagetoken={token}&key={API_KEY}"
            )
        else:
            break

    return resultados

# =========================================
# FUNCIÓN: Place Details
# =========================================
def obtener_detalles(place_id):
    url = (
        f"https://maps.googleapis.com/maps/api/place/details/json?"
        f"place_id={place_id}&fields=name,website,formatted_address,rating,geometry&key={API_KEY}"
    )
    resp = requests.get(url).json()
    return resp.get("result", {})

# =========================================
# EJECUCIÓN PRINCIPAL
# =========================================
resultado_final = []
seen_place_ids = set()

for prov, (lat_c, lng_c) in PROVINCES.items():
    print(f"\n🔍 Provincia: {prov}")

    for dx, dy in GRID_OFFSETS:
        lat = lat_c + dx
        lng = lng_c + dy
        print(f"  📍 Punto de búsqueda: {lat}, {lng}")

        for keyword in KEYWORDS:
            print(f"    🔹 Keyword: {keyword}")
            lugares = buscar_lugares(lat, lng, keyword)

            for lugar in lugares:
                place_id = lugar.get("place_id")
                if not place_id or place_id in seen_place_ids:
                    continue

                detalles = obtener_detalles(place_id)
                time.sleep(PAUSA_REQUEST)

                registro = {
                    "place_id": place_id,
                    "nombre": detalles.get("name") or lugar.get("name"),
                    "direccion": detalles.get("formatted_address") or lugar.get("vicinity"),
                    "provincia": prov,
                    "lat": detalles.get("geometry", {}).get("location", {}).get("lat"),
                    "lng": detalles.get("geometry", {}).get("location", {}).get("lng"),
                    "rating": detalles.get("rating"),
                    "website": detalles.get("website"),
                    "keyword_found": keyword
                }

                resultado_final.append(registro)
                seen_place_ids.add(place_id)

# =========================================
# GUARDAR JSON EN DRIVE
# =========================================
json_file = RUTA_DRIVE + "librerias_ecuador_pichincha_pastaza_intermedia.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(resultado_final, f, indent=4, ensure_ascii=False)

print(f"\n📁 Archivo JSON guardado en Drive: {json_file}")
print(f"Total registros únicos: {len(resultado_final)}")


🔍 Provincia: Pichincha
  📍 Punto de búsqueda: -0.21, -78.5
    🔹 Keyword: librería
    🔹 Keyword: venta de libros
    🔹 Keyword: libros
    🔹 Keyword: bookstore
    🔹 Keyword: book shop
    🔹 Keyword: papelería
    🔹 Keyword: material educativo
    🔹 Keyword: textos escolares
    🔹 Keyword: libros escolares
    🔹 Keyword: editorial
  📍 Punto de búsqueda: 0.09, -78.2
    🔹 Keyword: librería
    🔹 Keyword: venta de libros
    🔹 Keyword: libros
    🔹 Keyword: bookstore
    🔹 Keyword: book shop
    🔹 Keyword: papelería
    🔹 Keyword: material educativo
    🔹 Keyword: textos escolares
    🔹 Keyword: libros escolares
    🔹 Keyword: editorial
  📍 Punto de búsqueda: -0.51, -78.8
    🔹 Keyword: librería
    🔹 Keyword: venta de libros
    🔹 Keyword: libros
    🔹 Keyword: bookstore
    🔹 Keyword: book shop
    🔹 Keyword: papelería
    🔹 Keyword: material educativo
    🔹 Keyword: textos escolares
    🔹 Keyword: libros escolares
    🔹 Keyword: editorial

🔍 Provincia: Pastaza
  📍 Punto de búsqueda:

# Hacemos Web Scraping de las paginas

In [ ]:
import json
import time
import re
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import *
from webdriver_manager.chrome import ChromeDriverManager
from google.colab import drive

# ====== MONTAR GOOGLE DRIVE ======
DRIVE_PATH = "/content/drive/MyDrive/LibreriasEcuador"  # Cambia si quieres otra carpeta

import os
os.makedirs(DRIVE_PATH, exist_ok=True)

# ====== CONFIG ======
Num_Enlaces = 100
JSON_INPUT = "/content/drive/MyDrive/LibreriasEcuador/librerias_ecuador_pichincha_pastaza_intermedia.json"

# ====== FUNCIONES ======
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-plugins")
    chrome_options.add_argument("--disable-images")
    chrome_options.add_argument("--lang=es")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => false});")
    driver.set_page_load_timeout(25)
    return driver

def extract_text_from_page(driver, url):
    try:
        driver.get(url)
        time.sleep(4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        for tag in soup(["script","style","nav","footer","header","aside","form","noscript","svg"]):
            tag.decompose()
        text = soup.get_text(separator='\n')
        lines = [line.strip() for line in text.splitlines() if line.strip()]
        chunks = [phrase.strip() for line in lines for phrase in line.split("  ") if phrase.strip()]
        clean_text = '\n'.join(chunks)
        return clean_text[:15000]
    except Exception as e:
        return f"[ERROR: {str(e)}]"

def get_internal_links(driver, base_url, max_links=15):
    try:
        driver.get(base_url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        links = set()
        domain = urlparse(base_url).netloc
        for a in soup.find_all('a', href=True):
            href = a['href'].split('#')[0].strip()
            if not href or href.startswith(('tel:', 'mailto:', 'whatsapp:', 'javascript:')):
                continue
            full_url = urljoin(base_url, href)
            if urlparse(full_url).netloc == domain and full_url.startswith('http'):
                if any(ext in full_url.lower() for ext in ['.pdf', '.jpg', '.png', '.zip', '.doc', '.exe']):
                    continue
                links.add(full_url)
        return list(links)[:max_links]
    except:
        return []

def scrape_full_website(website_url, max_pages):
    if not website_url or not website_url.startswith('http'):
        return {"error": "URL inválida"}
    driver = None
    try:
        driver = get_driver()
        print(f"Scraping: {website_url}")
        main_text = extract_text_from_page(driver, website_url)
        all_text = {"home": main_text}
        links = get_internal_links(driver, website_url)
        print(f"  Enlaces internos: {len(links)}")
        for i, link in enumerate(links[:max_pages-1]):
            print(f"  → {i+1}: {link}")
            try:
                page_text = extract_text_from_page(driver, link)
                key = re.sub(r'\W+', '_', urlparse(link).path.strip('/')) or f"page_{i}"
                key = key[:50]
                all_text[key] = page_text
                time.sleep(2)
            except Exception as e:
                all_text[f"error_page_{i}"] = f"[Fallo en subpágina: {str(e)}]"
        return all_text
    except Exception as e:
        return {"error": f"Error general: {str(e)}"}
    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass

# ====== LEER JSON DE DRIVE ======
with open(JSON_INPUT, "r", encoding="utf-8") as f:
    data = json.load(f)

# ====== SCRAPING ======
results = []
all_text_output = ""

for negocio in data:
    website = negocio.get("website")
    nombre = negocio.get("nombre", "Sin nombre")

    print(f"\n{'='*60}")
    print(f"PROCESANDO: {nombre}")
    print(f"{'='*60}")

    if website and website.startswith("http"):
        scraped = scrape_full_website(website, max_pages=Num_Enlaces)
        results.append({"nombre": nombre, "website": website, "texto_extraido": scraped})

        all_text_output += f"\n\n{'='*60}\nNEGOCIO: {nombre}\nWEB: {website}\n{'='*60}\n"
        for k, v in scraped.items():
            all_text_output += f"\n--- {k.upper()} ---\n{v}\n"
    else:
        no_web = {"error": "No tiene sitio web"}
        results.append({"nombre": nombre, "website": website, "texto_extraido": no_web})
        all_text_output += f"\n\n=== {nombre} ===\nSIN SITIO WEB\n"

# ====== GUARDAR RESULTADOS EN DRIVE ======
json_output_path = os.path.join(DRIVE_PATH, "texto_scrapeado.json")
txt_output_path = os.path.join(DRIVE_PATH, "todo_el_texto.txt")

with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

with open(txt_output_path, "w", encoding="utf-8") as f:
    f.write(all_text_output)

print(f"\n¡SCRAPING COMPLETADO! Archivos guardados en Drive:\n{json_output_path}\n{txt_output_path}")

Streaming output truncated to the last 5000 lines.
  → 3: http://libreriajuridicaoni.online/?add-to-cart=12693
  → 4: http://libreriajuridicaoni.online/?add-to-cart=12768
  → 5: http://libreriajuridicaoni.online/?product-page=2
  → 6: http://libreriajuridicaoni.online/?product-page=4
  → 7: https://libreriajuridicaoni.online/producto/dolo-o-culpa-la-cuestion-de-la-culpa-temeraria/
  → 8: https://libreriajuridicaoni.online/categoria/todo/libreria/
  → 9: https://libreriajuridicaoni.online/50-anos-formando-mentes-juridicas-celebrando-el-aniversario-de-libreria-juridica-oni/
  → 10: https://libreriajuridicaoni.online/producto/teoria-general-del-uso-de-las-preguntas-en-el-litigio/
  → 11: http://libreriajuridicaoni.online/?add-to-cart=12746
  → 12: http://libreriajuridicaoni.online/?add-to-cart=12743
  → 13: http://libreriajuridicaoni.online/?add-to-cart=12690
  → 14: https://libreriajuridicaoni.online/tienda/
  → 15: https://libreriajuridicaoni.online/

PROCESANDO: Cogiler
Scraping: http:

# Agrupación de los links para eliminar datos duplicados

In [ ]:
import json, os
from tqdm import tqdm
from google.colab import drive


input_path = "/content/drive/MyDrive/LibreriasEcuador/texto_scrapeado.json"
output_path = "/content/drive/MyDrive/LibreriasEcuador/texto_scrapeado_unico.json"

# Cargar JSON original
with open(input_path, "r", encoding="utf-8") as f:
    librerias = json.load(f)

vistos = set()
agrupadas = []

for lib in tqdm(librerias, desc="Procesando librerías"):
    website = lib.get("website")  # usar get para evitar KeyError
    if not website:  # saltar si es None o cadena vacía
        continue
    website = website.strip()
    if website not in vistos:
        agrupadas.append(lib)
        vistos.add(website)

# Guardar JSON filtrado
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(agrupadas, f, indent=2, ensure_ascii=False)

print(f"✅ JSON filtrado guardado en: {output_path}")
print(f"Total librerías únicas por URL: {len(agrupadas)}")



Procesando librerías: 100%|██████████| 737/737 [00:00<00:00, 1525766.07it/s]

✅ JSON filtrado guardado en: /content/drive/MyDrive/LibreriasEcuador/texto_scrapeado_unico.json
Total librerías únicas por URL: 253


# Dividir en Chunks

In [ ]:
import json, os

# ============================================================
# 1. FUNCION PARA DIVIDIR EN CHUNKS
# ============================================================
def crear_chunks(input_path: str, output_path: str = "chunks_guardados.json", max_chars: int = 1500):
    with open(input_path, "r", encoding="utf-8") as f:
        librerias = json.load(f)

    todos_chunks = []

    for lib in librerias:
        nombre, website = lib["nombre"], lib["website"]

        for pagina, texto in lib["texto_extraido"].items():
            if "Cloudflare" in texto or len(texto) < 100:
                continue

            # Dividir en líneas
            lineas = [l.strip() for l in texto.split('\n') if l.strip()]
            chunks, chunk_actual, longitud = [], [], 0

            for linea in lineas:
                linea_salto = linea + "\n"
                if longitud + len(linea_salto) > max_chars and chunk_actual:
                    chunks.append({
                        "libreria": nombre,
                        "website": website,
                        "pagina": pagina,
                        "chunk_id": len(chunks),
                        "texto": "\n".join(chunk_actual)
                    })
                    chunk_actual = [linea]
                    longitud = len(linea_salto)
                else:
                    chunk_actual.append(linea)
                    longitud += len(linea_salto)

            if chunk_actual:
                chunks.append({
                    "libreria": nombre,
                    "website": website,
                    "pagina": pagina,
                    "chunk_id": len(chunks),
                    "texto": "\n".join(chunk_actual)
                })

            todos_chunks.extend(chunks)

    # Guardar chunks
    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(todos_chunks, f, indent=2, ensure_ascii=False)

    print(f"Chunks guardados: {len(todos_chunks)} → {output_path}")
    return todos_chunks

# ============================================================
# 2. EJECUTAR
# ============================================================
chunks = crear_chunks(
    "/content/drive/MyDrive/LibreriasEcuador/texto_scrapeado_unico.json",
    "/content/drive/MyDrive/LibreriasEcuador/chunks_guardados.json",
    max_chars=1500
)


Chunks guardados: 6472 → /content/drive/MyDrive/LibreriasEcuador/chunks_guardados.json


# Eliminar los datos ya analizados

In [ ]:
import json

# ============================================================
# 1. CARGAR ARCHIVOS
# ============================================================
chunks_path = "/content/drive/MyDrive/LibreriasEcuador/chunks_guardados.json"  # chunks crudos
procesados_path = "/content/drive/MyDrive/LibreriasEcuador/checkpoints/chk_https_www_mrbooks_com__collections_b1.json"  # libros procesados
output_pendientes = "/content/drive/MyDrive/LibreriasEcuador/chunks_pendientes.json"

with open(chunks_path, "r", encoding="utf-8") as f:
    chunks_crudos = json.load(f)

with open(procesados_path, "r", encoding="utf-8") as f:
    libros_procesados = json.load(f)

# ============================================================
# 2. OBTENER CHUNKS YA PROCESADOS
# ============================================================
# Creamos un set de tuplas únicas para identificar chunks procesados
# Ahora filtramos por (website, chunk_id)
chunks_procesados_set = set(
    (libro["website"], libro["chunk_id"])
    for libro in libros_procesados
)

# ============================================================
# 3. FILTRAR CHUNKS PENDIENTES
# ============================================================
chunks_pendientes = [
    chunk for chunk in chunks_crudos
    if (chunk["website"], chunk["chunk_id"]) not in chunks_procesados_set
]

# ============================================================
# 4. GUARDAR RESULTADO
# ============================================================
with open(output_pendientes, "w", encoding="utf-8") as f:
    json.dump(chunks_pendientes, f, indent=2, ensure_ascii=False)

print(f"Chunks pendientes: {len(chunks_pendientes)} → {output_pendientes}")



Chunks pendientes: 6198 → /content/drive/MyDrive/LibreriasEcuador/chunks_pendientes.json


# Pasamos el JSON con los Chunks pendientes a la IA para sacar los datos importantes segun el promt

In [ ]:
# ============================================================
# 1. INSTALAR + MONTAR + CARGAR MODELO
# ============================================================
!pip install -q transformers accelerate sentencepiece torch tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json, re, os, time
from tqdm import tqdm
from datetime import datetime
from google.colab import drive
import concurrent.futures

# ============================================================
# 2. CARGAR MODELO
# ============================================================
modelo_name = "Qwen/Qwen2.5-3B-Instruct"
print("\nCargando modelo Qwen2.5-3B...")
tokenizer = AutoTokenizer.from_pretrained(modelo_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    modelo_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()
print("Modelo listo")

# ============================================================
# 3. EXTRAER CATEGORÍA
# ============================================================
def extraer_categoria_del_chunk(chunk: str) -> str:
    chunk_lower = chunk.lower()
    categorias = [
        "más buscados", "novedades", "ofertas", "destacados",
        "más vendidos", "infantil", "juvenil", "literatura",
        "biografías", "ciencia ficción", "romance", "suspenso"
    ]
    for cat in categorias:
        if cat in chunk_lower:
            return cat.title()
    return "General"

# ============================================================
# 4. PROCESAR BATCH (SEGURA)
# ============================================================
def procesar_batch(chunks_batch, libreria, website, nombre_pagina):
    prompts, chunk_ids, categorias = [], [], []

    for i, chunk in chunks_batch:
        categoria = extraer_categoria_del_chunk(chunk)
        categorias.append(categoria)
        chunk_ids.append(i)

        prompt = f"""
Eres un extractor de libros.
Extrae TODOS los libros: TÍTULO + PRECIO ($ XX,XX)
Ignora valoraciones, botones, Cloudflare.
Si hay Cloudflare → responde []
Categoría = "{categoria}"
DEVUELVE SOLO JSON VÁLIDO

FORMATO:
[
  {{"titulo": "LIBRO", "precio": 10.00, "categoria": "{categoria}"}}
]

FRAGMENTO:
{chunk}
""".strip()
        prompts.append(prompt)

    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    respuestas = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    libros_batch = []

    def parsear_respuesta(idx):
        respuesta = respuestas[idx]
        json_start = respuesta.rfind('[')
        if json_start == -1:
            return []
        json_str = respuesta[json_start:]
        json_end = json_str.rfind(']')
        if json_end != -1:
            json_str = json_str[:json_end+1]
        json_str = re.sub(r"^```json\s*|```$", "", json_str, flags=re.MULTILINE).strip()

        try:
            libros = json.loads(json_str)
        except:
            return []

        libros_filtrados = []
        for libro in libros:
            try:
                titulo = str(libro.get("titulo", "")).strip()
                if not titulo:
                    continue
                precio = float(str(libro.get("precio", "0")).replace("$", "").replace(",", "."))
                categoria_libro = libro.get("categoria", categorias[idx])
            except:
                continue
            libros_filtrados.append({
                "titulo": titulo,
                "precio": precio,
                "categoria": categoria_libro,
                "libreria": libreria,
                "website": website,
                "pagina": nombre_pagina,
                "chunk_id": chunk_ids[idx],
                "extraido_en": datetime.now().strftime("%H:%M:%S")
            })
        return libros_filtrados

    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        resultados = list(executor.map(parsear_respuesta, range(len(prompts))))

    for idx, libros in enumerate(resultados):
        if libros:
            print(f"\nLIBROS DEL CHUNK {chunk_ids[idx]} | {categorias[idx]}:")
            for libro in libros:
                print(f"  • {libro['titulo']} | ${libro['precio']} | {libro['categoria']}")
        else:
            print(f"\n⚠️ Chunk {chunk_ids[idx]} no devolvió libros válidos")

    for r in resultados:
        libros_batch.extend(r)

    torch.cuda.empty_cache()
    return libros_batch

# ============================================================
# 5. PROCESAR TODO CON CHECKPOINTS
# ============================================================
def procesar_todo_con_chunks(input_path: str, output_json: str = "libros_finales.json", batch_size: int = 32, checkpoint_freq: int = 50):
    with open(input_path, "r", encoding="utf-8") as f:
        librerias = json.load(f)

    todos_libros = []
    start_time = time.time()
    checkpoint_dir = "/content/drive/MyDrive/LibreriasEcuador/checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)

    batch_contador_global = 0
    chunks_procesados = set()

    # Reanudar desde últimos checkpoints
    for file in sorted(os.listdir(checkpoint_dir)):
        if file.endswith(".json"):
            with open(os.path.join(checkpoint_dir, file), "r", encoding="utf-8") as f:
                data_chk = json.load(f)
                todos_libros.extend(data_chk)
                for libro in data_chk:
                    chunks_procesados.add((libro["website"], libro["chunk_id"]))

    for lib in tqdm(librerias, desc="Librerías"):
        nombre = lib.get("libreria") or lib.get("nombre")
        website = lib.get("website")
        pagina = lib.get("pagina")

        chunks = [(item["chunk_id"], item["texto"]) for item in lib.get("chunks", [lib])]
        chunks = [c for c in chunks if (website, c[0]) not in chunks_procesados]

        if not chunks:
            print(f"Todos los chunks de {nombre} → {pagina} ya fueron procesados.")
            continue

        for i in range(0, len(chunks), batch_size):
            batch_contador_global += 1
            batch_num = i // batch_size + 1
            safe_nombre = nombre.replace(" ", "_").replace("/", "_")
            safe_pagina = pagina.replace(" ", "_").replace("/", "_")

            batch = chunks[i:i+batch_size]
            print(f"\n{'='*60}")
            print(f"Procesando batch {batch_num}/{(len(chunks)-1)//batch_size + 1}")
            print(f"{nombre} → {pagina}")
            print(f"{'='*60}")

            libros = procesar_batch(batch, nombre, website, pagina)
            todos_libros.extend(libros)

            for chunk_id, _ in batch:
                chunks_procesados.add((website, chunk_id))

            print(f"→ {len(libros)} libros en este batch | Total: {len(todos_libros)}")

            if batch_contador_global % checkpoint_freq == 0:
                checkpoint_path = os.path.join(
                    checkpoint_dir,
                    f"checkpoint_{safe_nombre}_{safe_pagina}_batch_{batch_num}.json"
                )
                with open(checkpoint_path, "w", encoding="utf-8") as chk:
                    json.dump(todos_libros, chk, indent=2, ensure_ascii=False)
                print(f"   ✔ Checkpoint guardado cada {checkpoint_freq} batches: {checkpoint_path}")

    final_path = f"/content/drive/MyDrive/LibreriasEcuador/{output_json}"
    with open(final_path, "w", encoding="utf-8") as f:
        json.dump(todos_libros, f, indent=2, ensure_ascii=False)

    elapsed = time.time() - start_time
    print(f"\n{'='*60}")
    print(f"COMPLETO EN {elapsed:.1f} segundos | Total libros: {len(todos_libros)} → {final_path}")
    print(f"{'='*60}")
    return todos_libros

# ============================================================
# 6. EJECUTAR
# ============================================================
libros = procesar_todo_con_chunks(
    "/content/drive/MyDrive/LibreriasEcuador/chunks_pendientes.json",
    "libros_finales.json",
    batch_size=64,
    checkpoint_freq=10
)




Cargando modelo Qwen2.5-3B...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo listo


Librerías:   0%|          | 0/6198 [00:00<?, ?it/s]


Procesando batch 1/1
Librería Española 6 de Diciembre → categoria_producto_ecuador_biografias_personajes_c


...


LIBROS DEL CHUNK 8 | General:
  • LIBRO | $10.0 | General
→ 1 libros en este batch | Total: 1931

Procesando batch 1/1
MATYS GRAF → privacy_policy


Librerías: 100%|█████████▉| 6196/6198 [9:21:05<00:16,  8.25s/it]


⚠️ Chunk 9 no devolvió libros válidos
→ 0 libros en este batch | Total: 1931

Procesando batch 1/1
MATYS GRAF → privacy_policy


Librerías: 100%|██████████| 6198/6198 [9:21:18<00:00,  5.43s/it]


⚠️ Chunk 10 no devolvió libros válidos
→ 0 libros en este batch | Total: 1931
Todos los chunks de MATYS GRAF → lite ya fueron procesados.

COMPLETO EN 33678.4 segundos | Total libros: 1931 → /content/drive/MyDrive/LibreriasEcuador/libros_finales.json


# Eliminamos los datos que solo dicen Libro o Libro n o que no tienen precio o los mayores a 120

In [ ]:
import json
import re

def filtrar_libros(input_json, output_json):
    # Cargar el JSON
    with open(input_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    filtrados = []
    patron = re.compile(r"^libro(\s*\d+)?$", re.IGNORECASE)

    for libro in data:
        titulo = libro.get("titulo", "").strip()

        # ❌ ELIMINAR SI PRECIO == 0
        if libro.get("precio", 0) == 0:
            continue

        if libro.get("precio", 0) > 120:
            continue

        # ❌ ELIMINAR SI TITULO ES "LIBRO" O "LIBRO 5"
        if patron.match(titulo):
            continue

        filtrados.append(libro)

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(filtrados, f, ensure_ascii=False, indent=2)

    print(f"📦 Libros filtrados guardados en {output_json}")
    print(f"🔢 Total antes: {len(data)} | Total después: {len(filtrados)}")


# ✅ Ahora sí lo llamas
filtrar_libros(
    "/content/drive/MyDrive/LibreriasEcuador/libros_finales.json",
    "/content/drive/MyDrive/LibreriasEcuador/libros_depurados.json"
)



📦 Libros filtrados guardados en /content/drive/MyDrive/LibreriasEcuador/libros_depurados.json
🔢 Total antes: 1931 | Total después: 1067


# Unimos la información

In [ ]:
import json

def combinar_por_website(json_libros, json_librerias, output_json):
    with open(json_libros, "r", encoding="utf-8") as f:
        libros = json.load(f)

    with open(json_librerias, "r", encoding="utf-8") as f:
        librerias = json.load(f)

    # 🛑 Crear diccionario solo con librerías que SÍ tienen website válido
    librerias_dict = {}
    for lib in librerias:
        web = lib.get("website")
        if isinstance(web, str) and web.strip():   # Validar que sea string y no esté vacío
            librerias_dict[web.strip().lower()] = lib

    print(f"🔍 Librerías con website válido: {len(librerias_dict)}/{len(librerias)}")

    combinados = []

    for libro in libros:
        web = libro.get("website", "").strip().lower()

        if web in librerias_dict:
            nuevo = {**librerias_dict[web], **libro}
            combinados.append(nuevo)
        else:
            combinados.append(libro)

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(combinados, f, ensure_ascii=False, indent=2)

    print("\n✅ COMBINACIÓN COMPLETADA")
    print(f"📚 Libros: {len(libros)}")
    print(f"🏪 Librerías: {len(librerias)}")
    print(f"🔗 Coincidencias: {len(combinados)}")
    print(f"📦 Guardado en: {output_json}")


combinar_por_website(
    "/content/drive/MyDrive/LibreriasEcuador/libros_depurados.json",
    "/content/drive/MyDrive/LibreriasEcuador/librerias_ecuador_pichincha_pastaza_intermedia.json",
    "/content/drive/MyDrive/LibreriasEcuador/librerias_procesadas.json"
)


🔍 Librerías con website válido: 253/737

✅ COMBINACIÓN COMPLETADA
📚 Libros: 1067
🏪 Librerías: 737
🔗 Coincidencias: 1067
📦 Guardado en: /content/drive/MyDrive/LibreriasEcuador/librerias_procesadas.json


# 📚 1 Contar libros repetidos entre librerías (popularidad)



In [2]:
import json
from collections import Counter, defaultdict

with open("/content/drive/MyDrive/LibreriasEcuador/librerias_procesadas.json", "r", encoding="utf-8") as f:
    data = json.load(f)

contador = Counter()
librerias_por_libro = defaultdict(set)  # para saber dónde aparece cada libro

for item in data:
    titulo = item["titulo"].strip().lower()
    libreria = item["nombre"]  # nombre del local
    contador[titulo] += 1
    librerias_por_libro[titulo].add(libreria)

print("\n📚 TOP libros más repetidos:\n")
for titulo, count in contador.most_common(20):
    print(f"{titulo.upper()} → {count} veces en {len(librerias_por_libro[titulo])} librerías")



📚 TOP libros más repetidos:

ULTIMOS DIAS EN BERLIN → 12 veces en 1 librerías
EL ULTIMO SECRETO → 12 veces en 1 librerías
HOY → 12 veces en 1 librerías
COMO CAZAR A TU ASESINO → 12 veces en 1 librerías
LA PROFESORA → 12 veces en 1 librerías
HOROSCOPO CHINO 2026 → 12 veces en 1 librerías
EL NUEVO VIAJE DE EL PRINCIPITO → 12 veces en 1 librerías
APUNTES PARA JOHN → 12 veces en 1 librerías
PERO QUE HAS HECHO → 12 veces en 1 librerías
LA NINA SALVAJE → 12 veces en 1 librerías
KRYPTONITA LA HISTORIETA → 12 veces en 1 librerías
DELIRIO → 12 veces en 1 librerías
NOCHE NEGRA → 12 veces en 1 librerías
EL CIRCULO DE LOS DIAS → 12 veces en 1 librerías
UNOS CUANTOS SUENOS → 12 veces en 1 librerías
DESPIERTA TU INTELIGENCIA → 10 veces en 1 librerías
EL CAMINO DEL ARTISTA PARA LOS PADRES → 10 veces en 1 librerías
DIARIO DE ANNE FRANK (ILUSTRADO) → 10 veces en 1 librerías
ANNA KADABRA 8 EL FESTIVAL DE BRUJERIA → 7 veces en 2 librerías
CUANDO LA VIDA TE CONFRONTA → 7 veces en 2 librerías


# 🏆 2️ Top N libros más vendidos / publicados

In [5]:
N = 10
top = contador.most_common(N)

print(f"\n🏆 TOP {N} LIBROS MÁS POPULARES\n")
for libro, repeticiones in top:
    print(f"{libro.upper()} → aparece {repeticiones} veces")



🏆 TOP 10 LIBROS MÁS POPULARES

ULTIMOS DIAS EN BERLIN → aparece 12 veces
EL ULTIMO SECRETO → aparece 12 veces
HOY → aparece 12 veces
COMO CAZAR A TU ASESINO → aparece 12 veces
LA PROFESORA → aparece 12 veces
HOROSCOPO CHINO 2026 → aparece 12 veces
EL NUEVO VIAJE DE EL PRINCIPITO → aparece 12 veces
APUNTES PARA JOHN → aparece 12 veces
PERO QUE HAS HECHO → aparece 12 veces
LA NINA SALVAJE → aparece 12 veces


# 💲 3️ Precio promedio por libro

In [6]:
from collections import defaultdict
import statistics

precios = defaultdict(list)

for item in data:
    precios[item["titulo"]].append(item["precio"])

print("\n💲 Precio promedio por libro:\n")
for titulo, valores in precios.items():
    print(f"{titulo} → promedio {round(statistics.mean(valores),2)} USD")



💲 Precio promedio por libro:

MISERY → promedio 14.9 USD
ARSÈNE LUPIN C, CABALLERO LADRÓN → promedio 8.2 USD
SOMBRA DEL VIENTO, LA -TD- → promedio 23.0 USD
FLORES EN EL ÁTIC (SAGA DOLLANGANGER 1) → promedio 18.0 USD
MI NOMBRE ES EMILIA DEL VALLE → promedio 23.5 USD
ENCUENTROS CON LIBROS → promedio 39.1 USD
REBELIÓN DE ATLAS, LA → promedio 26.42 USD
ELEMENTO & MOSAICO → promedio 6.0 USD
JARDÍN SOMBRIO (SAGA DOLLANGANGER 5) → promedio 15.5 USD
ANTE TODO NO HAGAS DAÑO -BOLSILLO- → promedio 15.9 USD
CUENTOS DE BEEDLE EL BARDO, LOS → promedio 25.49 USD
VERDAD Y JUSTIFICACIÓN → promedio 18.99 USD
ESPEJISMOS DE LA CERTEZA, LOS -REFLEXIONES SOBRE LA RELACIÓN ENTRE EL CUERPO Y LA MENTE- → promedio 34.99 USD
SOBREPESO EMOCIONAL → promedio 29.99 USD
SIRENA ESMERALDA Y EL TESORO PERDIDO → promedio 18.32 USD
CRETINOS, LOS -EL CURIOSO CUENTO DE CRETINOLANDIA- → promedio 14.66 USD
BLUEY NAVIDAD BALCON DE SANTA → promedio 14.99 USD
BLUEY DONDE ESTA BLUEY EN NAVIDAD → promedio 11.49 USD
DULCE NAVIDAD 

# 🏪 4️ Precio promedio por librería

In [7]:
libreria_precios = defaultdict(list)

for item in data:
    libreria_precios[item["nombre"]].append(item["precio"])

print("\n🏪 Precio promedio por librería:\n")
for lib, valores in libreria_precios.items():
    print(f"{lib} → {round(sum(valores)/len(valores),2)} USD")



🏪 Precio promedio por librería:

Librería Española CC La Esquina → 19.12 USD
Librería Mr.Books Mall El Jardín → 15.38 USD
Librería Mr.Books Scala Shopping → 16.81 USD
Librería The Owl → 22.25 USD
Elyon Books | Librería Cristiana ONLINE → 29.02 USD
Librería Studium - Plaza Mayor → 21.0 USD
Centro Cultural Casa Egüez → 120.0 USD
Fondo de cultura económica → 18.09 USD
Librería Rayuela → 19.95 USD
El Mundo Del Libro Mariana De Jesús → 22.99 USD
Librería San Pablo → 12.06 USD
Quicentro → 20.55 USD
Centro de vida nueva CVN → 14.74 USD
Lobolunar CCI → 24.39 USD
Quicentro Sur Shopping → 23.55 USD
El Mundo del Libro - Cotocollao → 22.99 USD
Libreria New Time → 29.35 USD
Supermercado AKI → 15.0 USD
Checks Comisariato Del Libro River Mall → 25.08 USD
LIBRERÍA EL GRAN LIBRO JURÍDICO → 24.35 USD
Spiritual Library Foundation Jesus of Mercy → 10.0 USD
Abya Yala Bookstore → 15.4 USD
Giving Tree Books → 16.15 USD
Cogiler → 26.68 USD
Megamaxi → 20.36 USD
Tía Tena → 406.89 USD
Coral Hipermercados - La Y

# 🏷️ 5️ Precio promedio por categoría

In [8]:
cat_precios = defaultdict(list)

for item in data:
    cat = item["categoria"]
    cat_precios[cat].append(item["precio"])

print("\n🏷️ Precio promedio por categoría:\n")
for cat, valores in cat_precios.items():
    print(f"{cat} → {round(sum(valores)/len(valores),2)} USD")



🏷️ Precio promedio por categoría:

Más Buscados → 19.4 USD
Destacados → 20.59 USD
General → 959.1 USD
Infantil → 28.65 USD
Juvenil → 21.86 USD
Novedades → 20.74 USD
Literatura → 19.24 USD
Romance → 19.0 USD
Ofertas → 28.96 USD
Más Vendidos → 8.93 USD
Relato Corto → 10.0 USD
Fantasía Oscura → 10.0 USD
Horror y Misterio → 10.0 USD
Microrrelatos → 10.0 USD
Relatos Destacados → 5.49 USD
Matemáticas → 36.0 USD


# 💥 6️ Detectar librerías más baratas (ranking de precio promedio)

In [9]:
ranking = sorted(libreria_precios.items(), key=lambda x: sum(x[1])/len(x[1]))

print("\n🥇 Librerías con precios más bajos:\n")
for nombre, precios in ranking[:10]:
    print(f"{nombre} → {round(sum(precios)/len(precios),2)} USD promedio")



🥇 Librerías con precios más bajos:

Mundo de Papel → 3.63 USD promedio
Lexus Editores en Ecuador - Lexus Kids → 6.28 USD promedio
PAPERLEN → 7.12 USD promedio
RelatoCorto.com → 8.5 USD promedio
Agencia Publicitaria - La Panadería Creativa - horneando ideas → 8.5 USD promedio
Ariel y Radmandí Publicaciones → 8.75 USD promedio
Project Papelería → 8.83 USD promedio
Spiritual Library Foundation Jesus of Mercy → 10.0 USD promedio
MEGATODO Bazar y Papelería → 10.0 USD promedio
PAPELERÍA - BPC → 10.0 USD promedio


# 🦄 7️ Detectar libros raros

In [3]:
from collections import Counter, defaultdict
import json

# Contar cuántas veces aparece cada libro
contador = Counter()
librerias_por_libro = defaultdict(list)

for item in data:
    titulo = item["titulo"].strip().lower()
    libreria = item["nombre"]
    precio   = item["precio"]
    provincia = item.get("provincia", "")
    contador[titulo] += 1

    librerias_por_libro[titulo].append({
        "libreria": libreria,
        "precio": precio,
        "provincia": provincia
    })

# Libros raros = aparecen 1 sola vez
raros = [titulo for titulo, count in contador.items() if count == 1]

print("\n🦄 LIBROS RAROS (solo 1 librería tiene este libro)\n")

for titulo in raros:
    info = librerias_por_libro[titulo][0]   # solo hay uno
    print(f"📘 {titulo.upper()}")
    print(f"   🏪 Librería: {info['libreria']}")
    print(f"   💲 Precio:  {info['precio']}")
    print(f"   📍 Provincia: {info['provincia']}\n")




🦄 LIBROS RAROS (solo 1 librería tiene este libro)

📘 MISERY
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  14.9
   📍 Provincia: Pichincha

📘 ARSÈNE LUPIN C, CABALLERO LADRÓN
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  8.2
   📍 Provincia: Pichincha

📘 SOMBRA DEL VIENTO, LA -TD-
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  23.0
   📍 Provincia: Pichincha

📘 FLORES EN EL ÁTIC (SAGA DOLLANGANGER 1)
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  18.0
   📍 Provincia: Pichincha

📘 MI NOMBRE ES EMILIA DEL VALLE
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  23.5
   📍 Provincia: Pichincha

📘 ENCUENTROS CON LIBROS
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  39.1
   📍 Provincia: Pichincha

📘 ELEMENTO & MOSAICO
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  6.0
   📍 Provincia: Pichincha

📘 JARDÍN SOMBRIO (SAGA DOLLANGANGER 5)
   🏪 Librería: Librería Española CC La Esquina
   💲 Precio:  15.5
  

# Extraemos las imagenes de los libros desde Google Book

In [ ]:
import json
import requests
from tqdm import tqdm

def obtener_imagen_google(nombre_libro):
    url = f"https://www.googleapis.com/books/v1/volumes?q={nombre_libro}"
    try:
        data = requests.get(url, timeout=10).json()
        return data["items"][0]["volumeInfo"]["imageLinks"]["thumbnail"]
    except:
        return None


def extraer_imagenes_desde_libros(input_json, output_json):
    # Cargar libros
    with open(input_json, "r", encoding="utf-8") as f:
        libros = json.load(f)

    # Obtener títulos únicos
    titulos_unicos = sorted({libro["titulo"].strip() for libro in libros})

    resultados = []

    print(f"📚 Consultando Google Books para {len(titulos_unicos)} libros...\n")

    for titulo in tqdm(titulos_unicos):
        imagen = obtener_imagen_google(titulo)

        resultados.append({
            "titulo": titulo,
            "imagen": imagen
        })

    # Guardar resultado
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(resultados, f, ensure_ascii=False, indent=2)

    print(f"\n📦 Guardado en {output_json}")
    print(f"🖼️ Imágenes encontradas: {sum(1 for r in resultados if r['imagen'])}/{len(resultados)}")


# ---------------------------------------------------
# 🔥 EJECUTAR
# ---------------------------------------------------

extraer_imagenes_desde_libros(
    "/content/drive/MyDrive/LibreriasEcuador/librerias_procesadas.json",
    "/content/drive/MyDrive/LibreriasEcuador/libros_imagenes.json"
)



📚 Consultando Google Books para 658 libros...



100%|██████████| 658/658 [09:54<00:00,  1.11it/s]


📦 Guardado en /content/drive/MyDrive/LibreriasEcuador/libros_imagenes.json
🖼️ Imágenes encontradas: 513/658
